In [ ]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
import sys
import sqlalchemy as sa

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    

load_dotenv()

In [ ]:
from keras.models import load_model

model_view = load_model('../model/c-view.keras')
model_cart = load_model('../model/c-cart.keras')
model_purchase = load_model('../model/c-purchase.keras')

In [ ]:
conn_string = 'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/cosmetic'.format(
    db_user=os.getenv('DB_USER'),
    db_password=os.getenv('DB_PASSWORD'),
    db_host=os.getenv('DB_HOST'),
    db_port=os.getenv('DB_PORT'),
)
engine = sa.create_engine(conn_string)

In [ ]:
import datetime

start = datetime.date(2020, 2, 10)
end = datetime.date(2020, 3, 1)

In [11]:
from collections import defaultdict
from sqlalchemy import orm
import random

from schemas.product import Product
from schemas.data_by_date import DataByDate
from utils.utils import date_range, data_descale, preprocess_c

prev_days = int(os.getenv('PREV_DAYS'))

with orm.Session(engine) as session, session.begin():
    products = dict[tuple[int, int], Product]()
    for row in session.scalars(sa.select(Product)).all():
        products[(row.product_id, row.category_id)] = row
    
    memorized = defaultdict(lambda: defaultdict(lambda: { 'view': 0, 'cart': 0, 'remove_from_cart': 0, 'purchase': 0 }))

    d = start - datetime.timedelta(days=max(30, prev_days))
    for i in range(max(30, prev_days)):
        for row in session.scalars(sa.select(DataByDate).where(DataByDate.date == d).order_by(DataByDate.rank.asc())):
            key = (row.product_id, row.category_id)
            memorized[d][key] = {
                'view': row.view,
                'cart': row.cart,
                'purchase': row.purchase,
            }

        d += datetime.timedelta(days=1)
    
    while d < end:
        print(d)
        records = list[dict]()

        rows = list[dict]()
        categories = defaultdict(lambda: list[dict]())
        for row in session.scalars(sa.select(DataByDate).where(DataByDate.date == d).order_by(DataByDate.rank.asc())):
            key = (row.product_id, row.category_id)
            record = {
                'product_id': row.product_id,
                'category_id': row.category_id,
                'old_rank_in_category': row.rank_in_category,
                'view': row.view,
                'cart': row.cart,
                'purchase': row.purchase,
                'rank_in_category': 0,
                'days_on_shelf': (d - products[key].release_date).days,
                'price': row.max_price,
            }

            rows.append(record)
            categories[row.category_id].append(record)

        
        def key_func(record: dict):
            key = (record['product_id'], record['category_id'])
            prev_sales = sum(memorized[p][key]['purchase'] for p in date_range(d - datetime.timedelta(days=30), d, datetime.timedelta(days=1)))
            prev_views = max(1, sum(memorized[p][key]['view'] for p in date_range(d - datetime.timedelta(days=30), d, datetime.timedelta(days=1))))
            return random.uniform(0, 1), record['days_on_shelf'], record['price']
        
        for category in categories.values():
            category.sort(key=key_func)
            for index, record in enumerate(category):
                record['rank_in_category'] = 1000
        
        for index, record in enumerate(rows):
            key = (record['product_id'], record['category_id'])

            for i in range(1, prev_days + 1):
                prev_d = d - datetime.timedelta(days=i)
                record[f'view_prev{i}'] = memorized[prev_d][key]['view']
                record[f'cart_prev{i}'] = memorized[prev_d][key]['cart']
                record[f'purchase_prev{i}'] = memorized[prev_d][key]['purchase']

            if (d - products[key].release_date).days >= 7:
                records.append(record)
            else:
                memorized[d][key] = {
                    'view': record['view'],
                    'cart': record['cart'],
                    'purchase': record['purchase'],
                }
        
        del memorized[d - datetime.timedelta(days=prev_days)]

        df = pd.DataFrame.from_records(records)
        result_df = df.loc[:,[
            'product_id', 
            'category_id',
            'old_rank_in_category',
            'rank_in_category',
            'view', 
            'cart', 
            'purchase', 
        ]]
        df = df.drop(columns=['product_id', 'category_id', 'old_rank_in_category'])
        X1, X2, y = preprocess_c(df)
        result_df['predicted_view'] = data_descale(model_view.predict([X1, X2])[:, 0], 'view')
        result_df['predicted_cart'] = data_descale(model_cart.predict([X1, X2])[:, 0], 'cart')
        result_df['predicted_purchase'] = data_descale(model_purchase.predict([X1, X2])[:, 0], 'purchase')

        
        name = d.strftime('%Y-%m-%d')
        result_df.to_csv(f'../.data/ranking/d-{name}.csv')
        for result in result_df.to_dict('records'):
            key = (result['product_id'], result['category_id'])
            memorized[d][key] = {
                'view': result['predicted_view'],
                'cart': result['predicted_cart'],
                'purchase': result['predicted_purchase'],
            }

        d += datetime.timedelta(days=1)        

2020-02-21


In [ ]:
from utils.utils import plot_accuracy

result_df = pd.concat([
    pd.read_csv(f'../.data/ranking/d-{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        start,
        end,
        datetime.timedelta(days=1)
    )
])


plot_accuracy(result_df['predicted_view'], result_df['view'], 'view')
plot_accuracy(result_df['predicted_cart'], result_df['cart'], 'cart')
plot_accuracy(result_df['predicted_purchase'], result_df['purchase'], 'purchase')